In [ ]:
# from google.colab import userdata
# ssh_password=userdata.get('SSH_PASSWORD')

In [ ]:
# # Install colab_ssh
# %pip install colab_ssh --upgrade

In [ ]:
# from colab_ssh import launch_ssh_cloudflared, init_git_cloudflared
# launch_ssh_cloudflared(password=ssh_password)

In [ ]:
# !conda run -n tts_env pip install TTS

In [ ]:
# CELL 2: INSTALL DEPENDENCIES (Fixed - Absolute Paths)
import os

print("🔍 Checking environment...")
if not os.path.exists("/content/.venv/bin/python"):
    print("⚠️ .venv missing! Recreating...")
    !python3.10 -m venv /content/.venv

print("📦 Installing Dependencies (Using Absolute Paths)...")

# 1. Install PyTorch (Explicit Path: /content/.venv/bin/pip)
# We removed "> /dev/null" so you can see if it actually works
!/content/.venv/bin/pip install torch==2.1.0 torchaudio==2.1.0 --index-url https://download.pytorch.org/whl/cu118

# 2. Install Coqui TTS
%cd /content/TTS
!/content/.venv/bin/pip install -e .

# 3. Install Project Requirements
%cd /content/tecGrwTechnical
!/content/.venv/bin/pip install pandas datasets soundfile librosa scipy

print("✅ Installation Complete!")

In [ ]:
# CELL 3: VERIFY SETUP
print("🔍 Verifying Audio Drivers...")

# Run verification using the direct path
!/content/.venv/bin/python -c "import torchaudio; import soundfile; print(f'✅ Torchaudio: {torchaudio.__version__}'); print(f'✅ Soundfile: {soundfile.__version__}')"

In [ ]:
# CELL 2: CLONE & INSTALL (Fixed with Absolute Paths)
import os
from google.colab import userdata

# --- 1. CLONE REPO ---
GITHUB_USERNAME = "MUKAMAFrancois"
REPO_NAME       = "tecGrwTechnical"
BRANCH          = "main"
GITHUB_TOKEN    = userdata.get('GITHUB_TOKEN')

repo_url = f"https://{GITHUB_TOKEN}@github.com/{GITHUB_USERNAME}/{REPO_NAME}.git"
project_dir = f"/content/{REPO_NAME}"

if os.path.exists(project_dir):
    print(f"🔄 Updating repo...")
    %cd {project_dir}
    !git pull origin {BRANCH}
else:
    print(f"📥 Cloning repo...")
    !git clone {repo_url} {project_dir}

In [ ]:
# --- 2. INSTALL DEPENDENCIES (Into Python 3.10 Venv) ---
print("⬇️ Cloning Coqui TTS Source...")
!git clone https://github.com/coqui-ai/TTS /content/TTS || echo "TTS already exists"

print("🛠️ Installing PyTorch & TTS (This takes ~3 mins)...")

# A. Install PyTorch 2.1 (Explicitly using /content/.venv/bin/pip)
!/content/.venv/bin/pip install torch==2.1.0 torchaudio==2.1.0 --index-url https://download.pytorch.org/whl/cu118

# B. Install Coqui TTS from Source
%cd /content/TTS
!/content/.venv/bin/pip install -e .

# C. Install Your Project Requirements
%cd {project_dir}
!/content/.venv/bin/pip install pandas datasets soundfile librosa scipy

print("✅ Installation Complete!")

In [ ]:
# CELL: FIX DEPENDENCIES
print("🛠️ Fixing library mismatch...")

# Downgrade 'datasets' to the stable 2.x version
!/content/.venv/bin/pip install "datasets==2.21.0"

print("✅ Fix applied! Now re-run the 'Load & Process Data' cell below.")

In [ ]:
# CELL 4: LOAD & PROCESS DATA
import os
import getpass

# Define project path
project_dir = "/content/tecGrwTechnical"
%cd {project_dir}

# Check if data is already processed
if not os.path.exists("/content/data/processed/metadata.csv"):
    print("⚙️ STARTING DATA PIPELINE...")

    # 1. Run Loader (Inspect Data)
    print("\n📊 Step 1: Running Loader...")
    # We use the absolute path to run loader.py
    !/content/.venv/bin/python loader.py

    # 2. Run Preprocessing (Create Training Files)
    print("\n⚙️ Step 2: Running Preprocessing...")
    # We use -m for src.preprocessing
    !/content/.venv/bin/python -m src.preprocessing

else:
    print("✅ Data is already processed and ready for training!")

In [ ]:
# CELL 5: TRAIN MODEL
import os

# Define environment variables
venv_python = "/content/.venv/bin/python"
project_dir = "/content/tecGrwTechnical"
output_dir = "/content/output"

# Create output directory explicitly (good practice)
os.makedirs(output_dir, exist_ok=True)

%cd {project_dir}

print("🚀 STARTING VITS TRAINING...")
print(f"   - Checkpoints will be saved to: {output_dir}")
print("   - This may take hours depending on epochs. Monitor the loss below.")

# Run the trainer module
!{venv_python} -m src.trainer

In [ ]:
# CELL: CREATE CORRECT INFERENCE SCRIPT
code = """
import sys
import os
import soundfile as sf
from TTS.utils.synthesizer import Synthesizer

# 1. Define Paths (Your specific training run)
output_folder = "/content/output/kin_vits_production-February-16-2026_11+30AM-9986128"
checkpoint_path = os.path.join(output_folder, "best_model_2040.pth")
config_path = os.path.join(output_folder, "config.json")
out_wav_path = "/content/final_result.wav"

print(f"🔄 Loading Synthesizer from: {output_folder}")

# 2. Initialize the Synthesizer
# This handles Model + Config + Tokenizer automatically
synthesizer = Synthesizer(
    tts_checkpoint=checkpoint_path,
    tts_config_path=config_path,
    use_cuda=True
)

# 3. Text to Speak
text = "Muraho, amakuru yawe? Nishimiye ko twavugana."
print(f"🗣️ Generating: '{text}'")

# 4. Generate Audio
# The synthesizer returns the raw audio data as a list/array
wav = synthesizer.tts(text)

# 5. Save to File
# We fetch the correct sample rate from the config (16000 Hz)
sample_rate = synthesizer.tts_config.audio.sample_rate
sf.write(out_wav_path, wav, sample_rate)

print(f"✅ Saved audio to: {out_wav_path}")
"""

# Write this code to the file
with open("/content/run_test.py", "w") as f:
    f.write(code)

print("✅ Created fixed inference script: /content/run_test.py")

In [ ]:
# CELL: RUN INFERENCE
print("🚀 Running Inference...")
!/content/.venv/bin/python /content/run_test.py

In [ ]:
# CELL: CHECK PLAYBACK SPEED
import IPython.display as ipd
import soundfile as sf

# Load the file we just made
data, samplerate = sf.read("/content/final_result.wav")

print(f"Stats: Rate={samplerate}Hz, Duration={len(data)/samplerate:.2f}s")
print("🎵 Playing at correct 16kHz speed:")
ipd.display(ipd.Audio(data, rate=16000)) # Force 16k